In [11]:
import pandas as pd

# Q1a

In [4]:
# Write a function that estimates the emission parameters from training set

In [5]:
 def emission_parameters_v1(training_filepath):
    with open(training_filepath, "r", encoding="UTF-8") as file:
        # Read the entire contents of the file
        training_contents = file.read().split("\n")
        
    # Convert the data into a Pandas Dataframe
    training_words = []
    training_labels = []
    print(training_contents)
    
    for line in training_contents:
        # Exclude spaces in the dataset in training
        if len(line) > 0:
            parts = line.split()  # Split the line by spaces
            # To account for RUS cases wher there are spaces in the words
            if len(parts) >= 3:    
                word = " ".join(parts[:-2])  # Join the parts to get the word
                label = parts[-1]            # The last part is the label
                training_words.append(word)
                training_labels.append(label)
            else:
                training_words.append(parts[0])
                training_words.append(parts[1])

#     # Create a pandas DataFrame
#     training_data = {
#         "x": training_words,
#         "y": training_labels
#     }

#     training_df = pd.DataFrame(training_data)
#     # print(training_df)

#    # Now we want to calculate the emission parameter for each word-label pair
#     emission_parameters = {}
    
#     for label in training_df["y"].unique()[0:7]:
#         label_df = training_df[training_df["y"] == label]
#         total_label_occurrences = len(label_df)
        
#         word_counts = label_df["x"].value_counts().to_dict()
#         emission_probabilities = {word: count / total_label_occurrences for word, count in word_counts.items()}
        
#         emission_parameters[label] = emission_probabilities
    
#     return emission_parameters
    

In [6]:
 def emission_parameters_v2(training_filepath):
    with open(training_filepath, "r", encoding="UTF-8") as file:
        # Read the entire contents of the file
        training_contents = file.read().split("\n")
        
    # Convert the data into a Pandas Dataframe
    training_words = []
    training_labels = []
    # print(training_contents)
    
    for line in training_contents:
        # Exclude spaces in the dataset in training
        if len(line) > 0:
            parts = line.split()  # Split the line by spaces
            print(parts)
            # To account for RUS cases wher there are spaces in the words
            if len(parts) >= 3:    
                word = " ".join(parts[:-2])  # Join the parts to get the word
                label = parts[-1]            # The last part is the label
                training_words.append(word)
                training_labels.append(label)
            else:
                training_words.append(parts[0])
                training_labels.append(parts[1])

    # Create a pandas DataFrame
    training_data = {
        "x": training_words,
        "y": training_labels
    }
    # print(len(training_words) ,len(training_labels))

    training_df = pd.DataFrame(training_data)

   # Now we want to calculate the emission parameter for each word-label pair
    emission_parameters = {}
    
    for label in training_df["y"].unique()[0:7]:
        label_df = training_df[training_df["y"] == label]
        total_label_occurrences = len(label_df)
        
        word_counts = label_df["x"].value_counts().to_dict()
        
        # Add #UNK# special word here with a count of 1 for ALL labels
        word_counts["#UNK#"] = 1
        
        # Update emission_probabilities denominator
        emission_probabilities = {word: count / (total_label_occurrences+1) for word, count in word_counts.items()}
        
        emission_parameters[label] = emission_probabilities
    
    return emission_parameters

def label_test_set(testing_filepath, emission_params):
    # Pulling out the test set
    with open(testing_filepath, "r", encoding="UTF-8") as file:
        # Read the entire contents of the file
        test_words = file.read().split('\n')
    
    labelled_sequence = []
    for word in test_words:
        
        # if len(word) == 0:
        #     labelled_sequence.append("Space")
        # else:
        #     max_emission_prob = 0.0
        #     for label, word_emission_probs in emission_params.items():
        #         if word in word_emission_probs:
        #             if word_emission_probs[word] > max_emission_prob:
        #                 max_emission_prob = word_emission_probs[word]
        #                 max_label = label
                        
        #         # Word does not exist in the dataset
        #         else:
        #             if word_emission_probs["#UNK#"] > max_emission_prob:
        #                 max_emission_prob = word_emission_probs["#UNK#"]
        #                 max_label = label

        #     labelled_sequence.append(max_label)
        
        if len(word) == 0:
            labelled_sequence.append("Space")
        else:
            # Check if word has already been trained in the dataset
            exists = False
            
            for label, word_emission_probs in emission_params.items():
                if word in word_emission_probs:
                    exists = True
                    
            if exists:
                max_emission_prob = 0.0
                for label, word_emission_probs in emission_params.items():
                    if word in word_emission_probs:
                        if word_emission_probs[word] > max_emission_prob:
                            max_emission_prob = word_emission_probs[word]
                            max_label = label
            else:
                max_emission_prob = 0.0
                for label, word_emission_probs in emission_params.items():
                    if word_emission_probs["#UNK#"] > max_emission_prob:
                        max_emission_prob = word_emission_probs["#UNK#"]
                        max_label = label
            
            labelled_sequence.append(max_label)

    return labelled_sequence

def write_predictions_to_file(testing_filepath, test_labels, output_filepath):
    # Pulling out the test set
    with open(testing_filepath, "r", encoding="UTF-8") as file:
        # Read the entire contents of the file
        test_words = file.read().split('\n')
    with open(output_filepath, "w", encoding="UTF-8") as file:
        for word, label in zip(test_words, test_labels):
            if label == "Space":
                file.write("\n")
            else:
                file.write(f"{word} {label}\n")

In [7]:
# ES Dataset Prediction
training_emission_params_v2 = emission_parameters_v2(training_filepath="Data/ES/train")
predicted_labels = label_test_set(testing_filepath="Data/ES/dev.in", emission_params=training_emission_params_v2)
write_predictions_to_file(testing_filepath="Data/ES/dev.in", test_labels=predicted_labels,output_filepath="Data/ES/dev.p1.out")

['Estuvimos', 'O']
['hace', 'O']
['poco', 'O']
['mi', 'O']
['pareja', 'O']
['y', 'O']
['yo', 'O']
['comiendo', 'O']
['y', 'O']
['resultó', 'O']
['todo', 'O']
['muy', 'O']
['bien', 'O']
[',', 'O']
['tanto', 'O']
['la', 'O']
['comida', 'B-positive']
[',', 'O']
['el', 'O']
['vino', 'B-positive']
[',', 'O']
['el', 'O']
['trato', 'B-positive']
[',', 'O']
['la', 'O']
['decoración', 'B-positive']
['…', 'O']
['nos', 'O']
['gustó', 'O']
['todo', 'O']
['mucho', 'O']
['.', 'O']
['Por', 'O']
['poner', 'O']
['algún', 'O']
['pero', 'O']
[',', 'O']
['quizá', 'O']
['el', 'O']
['jamón', 'B-negative']
['no', 'O']
['era', 'O']
['todo', 'O']
['lo', 'O']
['"', 'O']
['ibérico', 'O']
['"', 'O']
['que', 'O']
['cabía', 'O']
['esperar', 'O']
['.', 'O']
['Bien', 'O']
['lo', 'O']
['sabe', 'O']
['el', 'O']
['autor', 'O']
['del', 'O']
['blog', 'O']
['.', 'O']
[')', 'O']
['Comida', 'B-positive']
['exquisita', 'O']
['.', 'O']
['Restaurante', 'B-positive']
['diferente', 'O']
[',', 'O']
['creativo', 'O']
['y', 'O']
['a

In [8]:
# Now perform ES evaluation with given script
!python EvalScript/evalResult.py Data/ES/dev.out Data/ES/dev.p1.out


#Entity in gold data: 229
#Entity in prediction: 1466

#Correct Entity : 178
Entity  precision: 0.1214
Entity  recall: 0.7773
Entity  F: 0.2100

#Correct Sentiment : 97
Sentiment  precision: 0.0662
Sentiment  recall: 0.4236
Sentiment  F: 0.1145


In [9]:
# RU Dataset Prediction
training_emission_params_v2 = emission_parameters_v2(training_filepath="Data/RU/train")
predicted_labels = label_test_set(testing_filepath="Data/RU/dev.in", emission_params=training_emission_params_v2)
write_predictions_to_file(testing_filepath="Data/RU/dev.in", test_labels=predicted_labels,output_filepath="Data/RU/dev.p1.out")

['Еда', 'B-positive']
['вкусная', 'O']
[',', 'O']
['но', 'O']
['отдельно', 'O']
['хочу', 'O']
['отметить', 'O']
['красивую', 'O']
['сервировку', 'B-positive']
['блюд', 'I-positive']
[';', 'O']
['.', 'O']
['Филадельфию', 'B-positive']
['мне', 'O']
['удалось', 'O']
['только', 'O']
['попробовать', 'O']
[',', 'O']
['сделали', 'O']
['на', 'O']
['отлично', 'O']
[',', 'O']
['хороший', 'O']
['кусок', 'O']
['лосося-филадельфии-чуток', 'B-positive']
['риса', 'I-positive']
[',', 'O']
['смело', 'O']
['возьму', 'O']
['в', 'O']
['следующий', 'O']
['раз', 'O']
['Десерт', 'B-positive']
['Тирамису', 'I-positive']
['просто', 'O']
['таял', 'O']
['во', 'O']
['рту', 'O']
[')', 'O']
[')', 'O']
[')', 'O']
['Очень', 'O']
['благодарны', 'O']
['персоналу', 'B-positive']
['за', 'O']
['качественное', 'O']
['обслуживание', 'B-positive']
['.', 'O']
['Были', 'O']
['здесь', 'O']
['первый', 'O']
['раз', 'O']
['и', 'O']
[',', 'O']
['точно', 'O']
[',', 'O']
['не', 'O']
['последний', 'O']
['!', 'O']
['Резервировать', 'O'

In [10]:
!python EvalScript/evalResult.py Data/RU/dev.out Data/RU/dev.p1.out


#Entity in gold data: 389
#Entity in prediction: 1816

#Correct Entity : 266
Entity  precision: 0.1465
Entity  recall: 0.6838
Entity  F: 0.2413

#Correct Sentiment : 129
Sentiment  precision: 0.0710
Sentiment  recall: 0.3316
Sentiment  F: 0.1170
